# Compute portfolio WARF

The following case-study demonstrates how to compute the weighted average rating
factor for a portfolio.

## Preliminary tasks
As a first step, we are going to import a portfolio into a `pd.DataFrame`. We'll call
it `port_df`. This dataframe comprises a number of securities with respective weights
 and ratings.

In [ ]:
import pandas as pd

import pyratings as rtg

port_df = pd.read_excel("portfolio.xlsx", sheet_name="long_term_ratings_worst")
port_df.head()

## Computing portfolio WARF
The column <i>worst_rtg</i> will be used in order to translate the ratings into WARFs
according to the following table.

![Long-term rating scale WARF](/../media/long_term_rating_scales_warf.png)

We will use
[``get_warf_from_ratings``](../../api_reference/#pyratings.get_warf.get_warf_from_ratings)
to translate the human-readable ratings into WARFs. The function needs a
rating provider (here: "S&P") in order to select an appropriate translation dictionary.

In [ ]:
port_warf_df = pd.concat(
    [
        port_df,
        rtg.get_warf_from_ratings(
            ratings=port_df["worst_rtg"],
            rating_provider="S&P",
        )
    ],
    axis=1
)

port_warf_df

Finally, we need to compute the WARF on a portfolio basis, which we subsequently
convert back into a human-readable rating. For the former, we use the
[``get_weighted_average``](../../api_reference/#pyratings.aggregate.get_weighted_average)
function and the
[``get_ratings_from_warf``](../../api_reference/#pyratings.get_ratings.get_ratings_from_warf)
function for the latter.

In [ ]:
avg_warf = rtg.get_weighted_average(
    data=port_warf_df["warf_worst_rtg"],
    weights=port_warf_df["weight"] / 100,
)

print(f"WARF: {avg_warf:.2f}")

In [ ]:
avg_warf_equivalent_rating = rtg.get_ratings_from_warf(
    warf=avg_warf,
    rating_provider="SP",
)

print(f"Portfolio equivalent WARF rating: {avg_warf_equivalent_rating}")

There you go. The portfolio WARF is A-.

Do you prefer to show the rating using Moody’s rating scale? — It’s as simple as
changing the ``rating_provider`` argument to “Moody”.

In [ ]:
avg_warf_equivalent_rating = rtg.get_ratings_from_warf(
    warf=avg_warf,
    rating_provider="Moody"
)

print(f"Portfolio equivalent WARF rating: {avg_warf_equivalent_rating}")

## Compute the WARF buffer
The WARF buffer is the distance from the current WARF to the next ``maxWARF`` level. It
determines the room until a further rating downgrade takes place.

Earlier, we determined that the portfolio WARF is equal to 165.58. Let's
compute the WARF buffer.

The corresponding ``maxWARF`` value is 220. The buffer is then equal to 220 - 165.58
= 54.42.

**pyratings** provides the function
[``get_warf_buffer``](../../api_reference/#pyratings.warf.get_warf_buffer) to help
you compute the WARF buffer.


In [ ]:
print(f"WARF buffer: {rtg.get_warf_buffer(avg_warf):.2f}")